# Download Historical Streamflow Data

This Jupyter Notebook serves as the dedicated module for retrieving historical streamflow data, a pivotal component for the calibration and evaluation of the hydrological models in this study. Leveraging the capabilities of the `dataRetrieval` R package, designed for accessing U.S. Geological Survey (USGS) water data, the notebook systematically downloads streamflow records for all active and relevant stream gauge stations identified within the previously defined study region. The acquired streamflow time series represent the observed hydrographs, which are indispensable for calibrating the ModClark model's parameters ($T_c$ and $R$) and for rigorously assessing the predictive performance of the machine learning models by comparing their generated hydrographs against these real-world observations.

This notebook:

1. Download streamflow data
2. Cleans streamflow data
3. Salves cleaned file locally
4. Gets Stream Gauge Station info (including geo location)
5. Salves Steam Gauge Station info locally

### Load the streamflow station data

In [1]:
# List of required packages
required_packages <- c("sf", "dataRetrieval", "dplyr", "arrow", "IRdisplay")

# Function to check and install packages
check_and_install <- function(package) {
  if (!require(package, character.only = TRUE)) {
    install.packages(package, dependencies = TRUE)
    library(package, character.only = TRUE)
  }
}

# Apply the function to each package
lapply(required_packages, check_and_install)    

Carregando pacotes exigidos: sf

Linking to GEOS 3.8.1, GDAL 3.2.1, PROJ 7.2.1

Carregando pacotes exigidos: dataRetrieval

Carregando pacotes exigidos: dplyr


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Carregando pacotes exigidos: arrow


Attaching package: 'arrow'


The following object is masked from 'package:utils':

    timestamp


Carregando pacotes exigidos: IRdisplay



[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

In [7]:
# Load the .gpkg file
gpkg_file <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/geo/gpkg/study_area_stream_gauge_stn.gpkg"
stations <- st_read(gpkg_file)
glimpse(stations)

Reading layer `study_area_stream_gauge_stn' from data source 
  `/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/geo/gpkg/study_area_stream_gauge_stn.gpkg' 
  using driver `GPKG'
Simple feature collection with 1056 features and 14 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -89.16257 ymin: 35.70829 xmax: -78.16294 ymax: 42.15683
Geodetic CRS:  NAD83
Rows: 1,056
Columns: 15
$ STAID      <chr> "03165000", "03164000", "03165500", "03161000", "03208500",~
$ STANAME    <chr> "CHESTNUT CREEK AT GALAX, VA", "NEW RIVER NEAR GALAX, VA", ~
$ ST         <chr> "va", "va", "va", "nc", "va", "va", "va", "ky", "va", "va",~
$ HUC        <chr> "05050001", "05050001", "05050001", "05050001", "05070202",~
$ CLASS      <int> 5, 0, 6, 5, 7, 8, 0, 0, 6, 6, 6, 0, 6, 7, 7, 0, 0, 0, 7, 0,~
$ FLOW       <dbl> 59.4, 0.0, 3690.0, 429.0, 1490.0, 2340.0, 0.0, 0.0, 5550.0,~
$ STAGE      <dbl> 1.56, 2.18, 4.26, 2.48, 4.60, 6.04, 11.11, 10.01, 3.05, 2.9~
$ TIME       <chr> "2

### Download Streamflow  Historical Data

The time span chosen for this study is 10 years. Therefore data will be downloaded from **01-01-2014 to 05-08-2025**.

In [6]:
# Define function to download 15-minute streamflow data
download_usgs_streamflow <- function(site_numbers, start_date, end_date, parameter_cd = "00060") {
  # Input validation
  if (!is.character(site_numbers) || length(site_numbers) == 0) {
    stop("site_numbers must be a non-empty character vector of USGS site IDs")
  }
  if (!is.character(start_date) || !is.character(end_date)) {
    stop("start_date and end_date must be character strings in YYYY-MM-DD format")
  }

  # Download 15-minute streamflow data using dataRetrieval
  # parameter_cd = "00060" is for discharge (cubic feet per second)
  streamflow_data <- readNWISuv(
    siteNumbers = site_numbers,
    parameterCd = parameter_cd,
    startDate = start_date,
    endDate = end_date,
    tz = "UTC"  # Time zone for data retrieval
  )

  # Check if data was retrieved
  if (nrow(streamflow_data) == 0) {
    # warning("No data retrieved for the specified sites and date range")
    return(NULL)
  }
  return(streamflow_data)
}


In [7]:
# Define function to save data locally as Parquet
save_streamflow_data <- function(data, site_no, output_dir) {
  # Drop redundant column stite_no (this number is incorporated in the file name)
  # Create output directory if it doesn't exist
  if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE)
  }
  
  # Check if data is valid
  if (is.null(data) || nrow(data) == 0) {
    # warning(paste("No data to save for site", site_no))
    return(FALSE)
  }
  
  # Define file path for Parquet
  file_path <- file.path(output_dir, paste0("streamflow_", site_no, ".parquet"))
  
  # Save data as Parquet
  write_parquet(data, file_path)
  # print(paste("Data saved for site", site_no, "at", file_path))
  return(TRUE)
}

**Download one station data as example**

In [13]:
station_code_example <- stations$STAID[1]

# Define date range
start_date <- "2014-01-01"
end_date <- "2014-05-08"

# Download data example
print(paste("Downloading data for station ID", station_code_example))
streamflow_data_example <- download_usgs_streamflow(
    site_numbers = station_code_example,
    start_date = start_date,
    end_date = end_date
)
glimpse(streamflow_data_example)
head(streamflow_data_example)


[1] "Downloading data for station ID 03165000"


GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03165000&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2014-05-08



Rows: 9,404
Columns: 6
$ agency_cd        <chr> "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "USGS~
$ site_no          <chr> "03165000", "03165000", "03165000", "03165000", "0316~
$ dateTime         <dttm> 2014-01-14 05:00:00, 2014-01-14 05:15:00, 2014-01-14~
$ X_00060_00000    <dbl> 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.~
$ X_00060_00000_cd <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A"~
$ tz_cd            <chr> "UTC", "UTC", "UTC", "UTC", "UTC", "UTC", "UTC", "UTC~


,agency_cd,site_no,dateTime,X_00060_00000,X_00060_00000_cd,tz_cd
,<chr>,<chr>,<dttm>,<dbl>,<chr>,<chr>
1,USGS,03165000,2014-01-14 05:00:00,101,A,UTC
2,USGS,03165000,2014-01-14 05:15:00,101,A,UTC
3,USGS,03165000,2014-01-14 05:30:00,101,A,UTC
4,USGS,03165000,2014-01-14 05:45:00,101,A,UTC
5,USGS,03165000,2014-01-14 06:00:00,101,A,UTC
6,USGS,03165000,2014-01-14 06:15:00,101,A,UTC


In [14]:
summary(streamflow_data_example)

  agency_cd           site_no             dateTime                  
 Length:9404        Length:9404        Min.   :2014-01-14 05:00:00  
 Class :character   Class :character   1st Qu.:2014-02-24 16:41:15  
 Mode  :character   Mode  :character   Median :2014-03-21 04:22:30  
                                       Mean   :2014-03-19 04:21:16  
                                       3rd Qu.:2014-04-14 16:03:45  
                                       Max.   :2014-05-09 03:45:00  
 X_00060_00000    X_00060_00000_cd      tz_cd          
 Min.   : 55.40   Length:9404        Length:9404       
 1st Qu.: 66.10   Class :character   Class :character  
 Median : 71.10   Mode  :character   Mode  :character  
 Mean   : 78.61                                        
 3rd Qu.: 79.80                                        
 Max.   :366.00                                        

#### Clean Downloaded Stream Flow Data**

The data retrieved from USGS, is pretty much clean already. However, to save space and time, after downloading each data file  from each station, unecessary columns will be removed, and columns renamed, before saving the file locally. The code  **00060** is used to retrieve discharge in **Cubic Feet per Second ($ft^3/s$)** - see documentation in (https://cran.r-project.org/web/packages/dataRetrieval/vignettes/dataRetrieval.htmlcolumn). The column name **X_00060_00000** indicates discharge ($ft^3/s$) and column **X_00060_00000_cd** is the measurement quality flag. **dateTime** column are in **UTC**. 

The cleaning steps are:
- Remove columns: `agency_cd`and `tz_cd`
- Rename columns: `X_00060_00000` and `X_00060_00000_cd`

Commom USGS parameter codes:
| pCode      | shortName |
| ----------- | ----------- |
| 00060	| Discharge [ft3/s] |
| 00065	| Gage height [ft] |
| 00010	| Temperature [C] |
| 00045	| Precipitation [in] |
| 00400	| pH |

In [8]:
  # Clean and format the data
clean_streamflow_data <- function(streamflow_data) {
  streamflow_data <- streamflow_data %>%
    select(site_no, dateTime, X_00060_00000, X_00060_00000_cd) %>%
    rename(
      discharge_cfs = X_00060_00000,
      discharge_quality = X_00060_00000_cd
    )
  
  return(streamflow_data)
}

In [16]:
cleaned_streamflow_data_example <- clean_streamflow_data(streamflow_data_example)
glimpse(cleaned_streamflow_data_example)
head(cleaned_streamflow_data_example)

Rows: 9,404
Columns: 4
$ site_no           <chr> "03165000", "03165000", "03165000", "03165000", "031~
$ dateTime          <dttm> 2014-01-14 05:00:00, 2014-01-14 05:15:00, 2014-01-1~
$ discharge_cfs     <dbl> 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101~
$ discharge_quality <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A~


,site_no,dateTime,discharge_cfs,discharge_quality
,<chr>,<dttm>,<dbl>,<chr>
1,03165000,2014-01-14 05:00:00,101,A
2,03165000,2014-01-14 05:15:00,101,A
3,03165000,2014-01-14 05:30:00,101,A
4,03165000,2014-01-14 05:45:00,101,A
5,03165000,2014-01-14 06:00:00,101,A
6,03165000,2014-01-14 06:15:00,101,A


#### Download data for over 1000 USGS station within the study area

In [ ]:
# Download data for each station using a for loop
station_codes <- stations$STAID

# Define date range for 10 years
start_date <- "2014-01-01"
end_date <- "2025-05-08"

# Define output directory
output_dir <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow"

# Initialize lists to store stations with errors and error messages
error_stations <- character()
error_messages <- character()

count <- 0
len <- length(station_codes)

# Loop through each station code
for (site in station_codes) {
  count <- count + 1
  
  # Check if file already exists
  file_path <- file.path(output_dir, paste0("streamflow_", site, ".parquet"))
  if (file.exists(file_path)) {
    # message(paste("File already exists for site", site, "- skipping download"))
    next
  }

  # Update status in notebook (overwrites previous output)
  display(paste0(count, "/", len, " - Downloading data for site ", site), display_id = "status", update = TRUE)
  
  
  # Try downloading data with error handling
  tryCatch({
    streamflow_data <- download_usgs_streamflow(
      site_numbers = site,
      start_date = start_date,
      end_date = end_date
    )
    
    # Save the data if download was successful
    if (!is.null(streamflow_data)) {
      streamflow_data <- clean_streamflow_data(streamflow_data)
    } else {
      # warning(paste("No data available for site", site))
      error_stations <- c(error_stations, site)
      error_messages <- c(error_messages, "No data available")
    }
    # Save the cleaned data
    save_streamflow_data(streamflow_data, site, output_dir)
    
    # Add a delay to avoid overwhelming the USGS server
    Sys.sleep(1)
  }, error = function(e) {
    # warning(paste("Error downloading data for site", site, ":", e$message))
    error_stations <- c(error_stations, site)
    error_messages <- c(error_messages, e$message)
  })
}

# Clear the status display
display("", display_id = "status", update = TRUE)

# Summary
message("Download and save process completed.")
message(paste("Check output directory:", output_dir))
if (length(error_stations) > 0) {
  message("Stations with errors: ", paste(error_stations, collapse = ", "))
  message("Error messages: ", paste(error_messages, collapse = "; "))
}

#### Failed Downloads

Not all stations could be downloaded. The likely cause, is that they might not have data covering the period of study **(2014 - 2025)**. This section investigate the failed downloads.

In [3]:
# Get the station IDs of downloaded streamflow data
streamflow_data_dir <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow"
streamflow_files <- list.files(path = streamflow_data_dir, full.names = FALSE)
downloaded_streamflow_STID <- gsub("streamflow_|\\.parquet", "", streamflow_files)
downloaded_streamflow_STID[1]

[1] "03007800"

In [4]:
# Get the station IDs of failed downloads
station_codes <- stations$STAID
failed_downloads_stid <- setdiff(station_codes, downloaded_streamflow_STID)
print(paste(length(failed_downloads_stid), "Station IDs failed to download"))
failed_downloads_stid[1:10]

[1] "350 Station IDs failed to download"


[1] "03170500"   "0317154954" "0317159760" "03178150"   "03178179"  
 [6] "03178000"   "03178120"   "03179800"   "03184500"   "0318989690"

In [9]:
# Function to get the period of data coverage for USGS stations
get_usgs_coverage <- function(site_numbers, parameter_cd = "00060", service = "uv") {
  # Input validation
  if (!is.character(site_numbers) || length(site_numbers) == 0) {
    stop("site_numbers must be a non-empty character vector of USGS site IDs")
  }
  if (!is.character(parameter_cd) || length(parameter_cd) != 1) {
    stop("parameter_cd must be a single character string (e.g., '00060' for discharge)")
  }
  if (!service %in% c("uv", "dv")) {
    stop("service must be 'uv' (instantaneous) or 'dv' (daily)")
  }
  
  # Query NWIS for data availability
  coverage_data <- tryCatch({
    whatNWISdata(
      siteNumber = site_numbers,
      parameterCd = parameter_cd,
      service = service
    )
  }, error = function(e) {
    warning(paste("Error querying NWIS for sites:", e$message))
    return(NULL)
  })
  
  # Check if data was retrieved
  if (is.null(coverage_data) || nrow(coverage_data) == 0) {
    warning("No coverage data available for the specified sites and parameter")
    return(data.frame(
      site_no = site_numbers,
      parameter_cd = parameter_cd,
      start_date = as.Date(NA),
      end_date = as.Date(NA),
      record_count = NA_integer_
    ))
  }
  return(coverage_data)
}

In [14]:
# Get converage for first two failed  sites
site_numbers <- failed_downloads_stid[1:2]
print(site_numbers)
coverage_data <- get_usgs_coverage(site_numbers, parameter_cd = "00060", service = "uv")
print(coverage_data)

[1] "03170500"   "0317154954"


GET: https://waterservices.usgs.gov/nwis/site/?seriesCatalogOutput=true&sites=03170500,0317154954&format=rdb

Warning message in get_usgs_coverage(site_numbers, parameter_cd = "00060", service = "uv"):
"No coverage data available for the specified sites and parameter"


     site_no parameter_cd start_date end_date record_count
1   03170500        00060       <NA>     <NA>           NA
2 0317154954        00060       <NA>     <NA>           NA


**Test the function to example station id of downloaded data**

In [25]:
# Get converage for first two downloaded five sites
site_numbers_examples_downloaded_data <- downloaded_streamflow_STID[1:5]
print(site_numbers_examples_downloaded_data)
coverage_downloaded_data <- get_usgs_coverage(site_numbers_examples_downloaded_data, parameter_cd = "00060", service = "uv")
print(coverage_downloaded_data)

[1] "03007800" "03010500" "03010655" "03010820" "03011020"


GET: https://waterservices.usgs.gov/nwis/site/?seriesCatalogOutput=true&sites=03007800,03010500,03010655,03010820,03011020&format=rdb



    agency_cd  site_no                           station_nm site_tp_cd
154      USGS 03007800 Allegheny River at Port Allegany, PA         ST
535      USGS 03010500        Allegheny River at Eldred, PA         ST
543      USGS 03010655     Oswayo Creek at Shinglehouse, PA         ST
550      USGS 03010820          ALLEGHENY RIVER AT OLEAN NY         ST
789      USGS 03011020      ALLEGHENY RIVER AT SALAMANCA NY         ST
    dec_lat_va dec_long_va coord_acy_cd dec_coord_datum_cd  alt_va alt_acy_va
154   41.81868   -78.29279            S              NAD83 1454.38       0.01
535   41.96340   -78.38613            S              NAD83 1416.53      11.00
543   41.96173   -78.19807            S              NAD83 1460.34       0.01
550   42.07236   -78.45086            1              NAD83 1399.50       0.01
789   42.15683   -78.71556            1              NAD83 1357.49       0.01
    alt_datum_cd   huc_cd data_type_cd parm_cd stat_cd  ts_id loc_web_ds
154       NAVD88 05010001        

In [26]:
parameter_cd = "00060"
# Select and format relevant columns
coverage_downloaded_data <- coverage_downloaded_data %>%
  dplyr::select(
    site_no,
    parm_cd,
    begin_date,
    end_date,
    count_nu
  ) %>%
  dplyr::rename(
    parameter_cd = parm_cd,
    start_date = begin_date,
    end_date = end_date,
    record_count = count_nu
  ) %>%
  dplyr::filter(parameter_cd == !!parameter_cd)

# Ensure all input sites are returned (even if no data)
all_sites <- data.frame(site_no = site_numbers_examples_downloaded_data)
coverage_downloaded_data <- all_sites %>%
  dplyr::left_join(coverage_downloaded_data, by = "site_no") %>%
  dplyr::mutate(
    parameter_cd = parameter_cd,
    start_date = as.Date(start_date),
    end_date = as.Date(end_date),
    record_count = as.integer(record_count)
  ) %>%
  dplyr::select(site_no, parameter_cd, start_date, end_date, record_count)
coverage_downloaded_data


site_no,parameter_cd,start_date,end_date,record_count
<chr>,<chr>,<date>,<date>,<int>
03007800,00060,1991-10-01,2025-05-11,12276
03010500,00060,1991-10-01,2025-05-11,12276
03010655,00060,1991-10-01,2025-05-11,12276
03010820,00060,2010-08-21,2025-05-11,5377
03011020,00060,1990-10-01,2025-05-11,12641


Apparently the two sation id in which it failed to dowload, have no data. The code below retrieves info about all failed stations. If no info is retrieved for any specific stations, it will be assumed no data is available for that station and will be excluded from modeling. 

In [37]:
# Define function to clean and format the coverage data
clean_usgs_coverage <- function(coverage_data, parameter_cd = "00060", site_numbers) {
  tryCatch({
    # Clean and format the coverage data
    coverage_data <- coverage_data %>%
      dplyr::select(
        site_no,
        parm_cd,
        begin_date,
        end_date,
        count_nu
      ) %>%
      dplyr::rename(
        parameter_cd = parm_cd,
        start_date = begin_date,
        end_date = end_date,
        record_count = count_nu
      ) %>%
      dplyr::filter(parameter_cd == !!parameter_cd)

    # Ensure all input sites are returned (even if no data)
    all_sites <- data.frame(site_no = site_numbers)
    coverage_data <- all_sites %>%
      dplyr::left_join(coverage_data, by = "site_no") %>%
      dplyr::mutate(
        parameter_cd = parameter_cd,
        start_date = as.Date(start_date),
        end_date = as.Date(end_date),
        record_count = as.integer(record_count)
      ) %>%
      dplyr::select(site_no, parameter_cd, start_date, end_date, record_count)

    return(coverage_data)
  }, error = function(e) {
    return(coverage_data)
  })
}

In [44]:
# Test the function for one failed site
site_number <- failed_downloads_stid[1]
parameter_cd <- "00060"
print(site_number)
coverage_data <- get_usgs_coverage(site_number, parameter_cd = parameter_cd, service = "uv") %>% 
  clean_usgs_coverage(parameter_cd = parameter_cd, site_numbers = site_number)
coverage_data


[1] "03170500"


GET: https://waterservices.usgs.gov/nwis/site/?seriesCatalogOutput=true&sites=03170500&format=rdb

Warning message in get_usgs_coverage(site_number, parameter_cd = parameter_cd, service = "uv"):
"No coverage data available for the specified sites and parameter"


site_no,parameter_cd,start_date,end_date,record_count
<chr>,<chr>,<date>,<date>,<int>
03170500,00060,NA,NA,NA


In [40]:
# Get coverage for all downloaded sites
site_numbers <- failed_downloads_stid
parameter_cd <- "00060"
coverage_data_all_failed_sites <- get_usgs_coverage(site_numbers, parameter_cd = parameter_cd, service = "uv") %>% 
  clean_usgs_coverage(parameter_cd = parameter_cd, site_numbers = site_numbers)
coverage_data_all_failed_sites

POST: https://waterservices.usgs.gov/nwis/site/



site_no,parameter_cd,start_date,end_date,record_count
<chr>,<chr>,<date>,<date>,<int>
03170500,NA,NA,NA,NA
0317154954,NA,NA,NA,NA
0317159760,NA,NA,NA,NA
03178150,NA,NA,NA,NA
03178179,NA,NA,NA,NA
03178000,00060,1997-02-14,1998-08-19,551
03178120,NA,NA,NA,NA
03179800,NA,NA,NA,NA
03184500,00060,1996-10-01,2003-09-30,2555


As seen from table above, the most of failed stations have no info data available. Next, is checked the period coverage for sites with avialable info. 

In [41]:
coverage_data_all_failed_sites %>% filter(!is.na(end_date)) %>% arrange(desc(end_date))

site_no,parameter_cd,start_date,end_date,record_count
<chr>,<chr>,<date>,<date>,<int>
03084800,00060,2024-10-01,2025-05-11,222
03085213,00060,2025-01-11,2025-05-11,120
03049819,00060,2025-01-11,2025-05-11,120
03049807,00060,2024-10-01,2025-05-11,222
03084698,00060,2007-10-01,2025-05-11,6432
03085947,00060,2007-10-01,2025-05-11,6432
03092090,00060,2022-10-01,2025-05-11,953
03229000,00060,2024-08-10,2025-05-11,274
03338780,00060,2007-10-01,2014-11-12,2599


There are 55 failed sites that actually have data available. Next is selected only the failed sites with data avialable in the coverage period (2014 - 2025), and attempt to download again data for those sites. 

In [42]:
coverage_data_sites_within_period_of_study <-  coverage_data_all_failed_sites %>%
 filter(!is.na(end_date) & !is.na(start_date)) %>%
 filter(end_date >= as.Date("2014-01-01")) %>%
 arrange(desc(end_date))
coverage_data_sites_within_period_of_study

site_no,parameter_cd,start_date,end_date,record_count
<chr>,<chr>,<date>,<date>,<int>
03084800,00060,2024-10-01,2025-05-11,222
03085213,00060,2025-01-11,2025-05-11,120
03049819,00060,2025-01-11,2025-05-11,120
03049807,00060,2024-10-01,2025-05-11,222
03084698,00060,2007-10-01,2025-05-11,6432
03085947,00060,2007-10-01,2025-05-11,6432
03092090,00060,2022-10-01,2025-05-11,953
03229000,00060,2024-08-10,2025-05-11,274
03338780,00060,2007-10-01,2014-11-12,2599


From the above, its seen there are 9 sites within the period of study. The `record_count` column however, seems to be quite small for the instateneous data (15 minute measurement). Next, is downloaded data for the stations shown in table above. 

In [49]:
# Define output directory
output_dir <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow"

count <- 0
len <- nrow(coverage_data_sites_within_period_of_study)

# Loop through each station code
for (i in (1:len)) {
  count <- count + 1
  site <- coverage_data_sites_within_period_of_study$site_no[i]
  start_date <- coverage_data_sites_within_period_of_study$start_date[i]
  if (start_date < as.Date("2014-01-01")) {
    start_date <- "2014-01-01"
  } else {
    start_date <- as.character(start_date)
  }
  end_date <- as.character(coverage_data_sites_within_period_of_study$end_date[i])

  # Check if file already exists
  file_path <- file.path(output_dir, paste0("streamflow_", site, ".parquet"))
  if (file.exists(file_path)) {
    message(paste("File already exists for site", site, "- skipping download"))
    next
  }

  # Update status in notebook (overwrites previous output)
  display(paste0(count, "/", len, " - Downloading data for site ", site), display_id = "status", update = TRUE)
  
  
  # Try downloading data with error handling
  tryCatch({
    streamflow_data <- download_usgs_streamflow(
      site_numbers = site,
      start_date = start_date,
      end_date = end_date
    )
    
    # Save the data if download was successful
    if (!is.null(streamflow_data)) {
      streamflow_data <- clean_streamflow_data(streamflow_data)
    } else {
      warning(paste("No data available for site", site))
    }
    # Save the cleaned data
    save_streamflow_data(streamflow_data, site, output_dir)
    
    # Add a delay to avoid overwhelming the USGS server
    Sys.sleep(1)
  }, error = function(e) {
    warning(paste("Error downloading data for site", site, ":", e$message))
  })
}

# Clear the status display
display("", display_id = "status", update = TRUE)

# Summary
message("Download and save process completed.")
message(paste("Check output directory:", output_dir))
if (length(error_stations) > 0) {
  message("Stations with errors: ", paste(error_stations, collapse = ", "))
  message("Error messages: ", paste(error_messages, collapse = "; "))
}

[1] "1/9 - Downloading data for site 03084800"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03084800&format=waterml,1.1&ParameterCd=00060&startDT=2024-10-01&endDT=2025-05-11

Warning message in value[[3L]](cond):
"Error downloading data for site 03084800 : "


[1] "2/9 - Downloading data for site 03085213"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03085213&format=waterml,1.1&ParameterCd=00060&startDT=2025-01-11&endDT=2025-05-11

Warning message in value[[3L]](cond):
"Error downloading data for site 03085213 : "


[1] "3/9 - Downloading data for site 03049819"

...

[1] "9/9 - Downloading data for site 03338780"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03338780&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2014-11-12

Warning message in value[[3L]](cond):
"Error downloading data for site 03338780 : "


[1] ""

Download and save process completed.

Check output directory: /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow



ERROR: Error in eval(expr, envir, enclos): objeto 'error_stations' n~ao encontrado


From above, all data failed to download. Therefore, **all 350** site in which download failed is excluded from the study. 

#### Checking for missing date

In the cleaning process, it should be checked for missing dates. Because it was not, next, missing dates are checked and filled. 

In [54]:
# List of downloaded streamflow files
streamflow_files <- list.files(path = streamflow_data_dir, full.names = TRUE)
streamflow_files[0:5]

[1] "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow/streamflow_03007800.parquet"
[2] "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow/streamflow_03010500.parquet"
[3] "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow/streamflow_03010655.parquet"
[4] "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow/streamflow_03010820.parquet"
[5] "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow/streamflow_03011020.parquet"

In [55]:
# Read example file
streamflow_data_example <- read_parquet(streamflow_files[1])
glimpse(streamflow_data_example)

Rows: 371,751
Columns: 4
$ site_no           <chr> "03007800", "03007800", "03007800", "03007800", "030~
$ dateTime          <dttm> 2014-01-01 05:00:00, 2014-01-01 05:15:00, 2014-01-0~
$ discharge_cfs     <dbl> 449, 449, 446, 446, 443, 443, 440, 437, 434, 432, 42~
$ discharge_quality <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A~


In [57]:
# Count the number of 15-minutes intervals of example data
package <- "lubridate"
# Function to check and install packages

if (!require(package, character.only = TRUE)) {
  install.packages(package, dependencies = TRUE)
  library(package, character.only = TRUE)
} else {
  library(package, character.only = TRUE)
}
  
count_15min_intervals <- function(date_min, date_max) {
  if (is.character(date_min)) {
    date_min <- as.POSIXct(date_min, format = "%Y%m%d %H:%M:%S", tz = "UTC")
  }
  if (is.character(date_max)) {
    date_max <- as.POSIXct(date_max, format = "%Y%m%d %H:%M:%S", tz = "UTC")
  }
  delta <- as.numeric(difftime(date_max, date_min, units = "mins"))
  total_intervals <- floor(delta / 15) + 1
  return(as.integer(total_intervals))
}

In [58]:
number_of_data_points <- count_15min_intervals(min(streamflow_data_example$dateTime), max(streamflow_data_example$dateTime))
number_of_15min_intervals_between_dates <- count_15min_intervals(min(streamflow_data_example$dateTime), max(streamflow_data_example$dateTime))
if (number_of_data_points == number_of_15min_intervals_between_dates) {
  message("The number of data points is equal to the number of 15-minute intervals. There is no missing dates")
} else {
  message(paste("The number of data points is NOT equal to the number of 15-minute intervals. There are", number_of_15min_intervals_between_dates-number_of_data_points, "missing dates"))
}

The number of data points is equal to the number of 15-minute intervals. There is no missing dates



In [61]:
# Check missing dates for all downloaded streamflow data
# Initiate empty list to store results
site_number <- c()
missing_dates <- c()
for (file in streamflow_files) {
    data <- read_parquet(file)
    site_number <- c(site_number, data$site_no[1])
    min_date <- min(data$dateTime)
    max_date <- max(data$dateTime)
    number_of_data_points <- count_15min_intervals(min_date, max_date)
    number_of_15min_intervals_between_dates <- count_15min_intervals(min_date, max_date)
    number_of_missing_dates <- number_of_15min_intervals_between_dates - number_of_data_points
    missing_dates <- c(missing_dates, number_of_missing_dates)
}
# Create a data frame with the results
missing_dates_df <- data.frame(
  site_no = site_number,
  missing_dates = missing_dates
)
head(missing_dates_df)

,site_no,missing_dates
,<chr>,<int>
1,03007800,0
2,03010500,0
3,03010655,0
4,03010820,0
5,03011020,0
6,03011800,0


In [62]:
# Check if any site has missing dates
missing_dates_df %>% filter(missing_dates > 0) %>% arrange(desc(missing_dates))

site_no,missing_dates
<chr>,<int>


**All site are complete in date  for the coverage period, not having any missing dates.**

#### Get and save station info

Station info (mainly geolocation info) is important to assure cohenrent CRS across os geolocation files. This and other info are retrieved from USGS for later preprocessing. 

In [6]:
# Modify cleaning function to include geolocation data
clean_usgs_coverage <- function(coverage_data, parameter_cd = "00060", site_numbers) {
  tryCatch({
    # Clean and format the coverage data
    coverage_data <- coverage_data %>%
      select(
        site_no,
        parm_cd,
        begin_date,
        end_date,
        count_nu,
        dec_lat_va,
        dec_long_va,
        dec_coord_datum_cd
      ) %>%
      rename(
        parameter_cd = parm_cd,
        start_date = begin_date,
        end_date = end_date,
        record_count = count_nu,
        latitude = dec_lat_va,
        longitude = dec_long_va,
        datum = dec_coord_datum_cd
      )

    # Ensure all input sites are returned (even if no data)
    all_sites <- data.frame(site_no = site_numbers)
    coverage_data <- all_sites %>%
      left_join(coverage_data, by = "site_no") %>%
      mutate(
        parameter_cd = parameter_cd,
        start_date = as.Date(start_date),
        end_date = as.Date(end_date),
        record_count = as.integer(record_count)
      )
    return(coverage_data)
  }, error = function(e) {
    return(coverage_data)
  })
}

In [10]:
# Get  streamflow info data for one site example
site_number <- stations$STAID[1]
example_station_info <- get_usgs_coverage(site_numbers=site_number)
example_station_info

GET: https://waterservices.usgs.gov/nwis/site/?seriesCatalogOutput=true&sites=03165000&format=rdb



,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,⋯,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>
59,USGS,03165000,"CHESTNUT CREEK AT GALAX, VA",ST,36.64596,-80.91924,U,NAD83,2344.8,0.1,⋯,NA,147650,NA,wat,NA,1645423,0,1990-10-01,2025-05-15,12645


In [11]:
# Clean streamflow info data site example
cleaned_example_station_info <- clean_usgs_coverage(example_station_info, parameter_cd = "00060", site_numbers = site_number)
cleaned_example_station_info

site_no,parameter_cd,start_date,end_date,record_count,latitude,longitude,datum
<chr>,<chr>,<date>,<date>,<int>,<dbl>,<dbl>,<chr>
03165000,00060,1990-10-01,2025-05-15,12645,36.64596,-80.91924,NAD83


In [12]:
# Download streamflow data for all sites with geolocation data
site_numbers <- stations$STAID
stations_info <- get_usgs_coverage(site_numbers = site_numbers)

POST: https://waterservices.usgs.gov/nwis/site/



In [13]:
head(stations_info)

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,⋯,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>
154,USGS,03007800,"Allegheny River at Port Allegany, PA",ST,41.81868,-78.29279,S,NAD83,1454.38,0.01,⋯,NA,122208,NA,wat,NA,1645423,0,1991-10-01,2025-05-15,12280
535,USGS,03010500,"Allegheny River at Eldred, PA",ST,41.96340,-78.38613,S,NAD83,1416.53,11.00,⋯,NA,122215,NA,wat,NA,1645423,0,1991-10-01,2025-05-15,12280
543,USGS,03010655,"Oswayo Creek at Shinglehouse, PA",ST,41.96173,-78.19807,S,NAD83,1460.34,0.01,⋯,NA,122220,NA,wat,NA,1645423,0,1991-10-01,2025-05-15,12280
593,USGS,03010820,ALLEGHENY RIVER AT OLEAN NY,ST,42.07236,-78.45086,1,NAD83,1399.50,0.01,⋯,NA,107518,NA,wat,NA,1645423,0,2010-08-21,2025-05-15,5381
836,USGS,03011020,ALLEGHENY RIVER AT SALAMANCA NY,ST,42.15683,-78.71556,1,NAD83,1357.49,0.01,⋯,NA,107522,NA,wat,NA,1645423,0,1990-10-01,2025-05-15,12645
1016,USGS,03011800,"Kinzua Creek near Guffey, PA",ST,41.76645,-78.71864,S,NAD83,1540.00,0.01,⋯,NA,122226,NA,wat,NA,1645423,0,1991-10-01,2025-05-15,12280


In [14]:
# Clean streamflow station info data
cleaned_stations_info <- clean_usgs_coverage(stations_info, parameter_cd = "00060", site_numbers = site_numbers)
head(cleaned_stations_info)

,site_no,parameter_cd,start_date,end_date,record_count,latitude,longitude,datum
,<chr>,<chr>,<date>,<date>,<int>,<dbl>,<dbl>,<chr>
1,03165000,00060,1990-10-01,2025-05-15,12645,36.64596,-80.91924,NAD83
2,03164000,00060,1990-10-01,2025-05-15,12645,36.64735,-80.97897,NAD83
3,03165500,00060,1996-01-25,2025-05-15,10703,36.83485,-80.95258,NAD83
4,03161000,00060,1985-10-01,2025-05-15,14471,36.39333,-81.40694,NAD83
5,03208500,00060,1990-10-01,2024-10-01,12419,37.20705,-82.29570,NAD83
6,03209000,00060,1990-10-01,2024-10-01,12419,37.23705,-82.34320,NAD83


In [15]:
# Save cleaned streamflow station info data
output_dir <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/tabular/streamflow"
# Create output directory if it doesn't exist
if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE)
}
# Define file path for Parquet
file_path <- file.path(output_dir, "streamflow_stations_info.parquet")
# Save data as Parquet
write_parquet(cleaned_stations_info, file_path)
# Print message
message(paste("Streamflow station info data saved at", file_path))

Streamflow station info data saved at /Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/tabular/streamflow/streamflow_stations_info.parquet

